In [ ]:
import json
import os
from tqdm import tqdm
from multiprocessing import Pool
import sys
import time
sys.path.append('/home/sreenath_a/Projects/OSS/Production/GitRipper')

import pandas as pd
from GitRipper.Ripper import collect

In [ ]:
cpu_count = os.cpu_count()
print(cpu_count)

In [ ]:
#Load the json file
with open('/home/sreenath_a/Projects/OSS/API_keys.json') as f:
    keys = json.load(f)['keys']

In [ ]:
columns = ['repo_owner', 'repo_name', 'oid', 'messageHeadline', 'author_name',
       'author_email', 'author_user_login', 'author_user_location',
       'author_user_company', 'author_user_pronouns', 'author_user_bio',
       'author_user_websiteUrl', 'author_user_twitterUsername', 'author_date',
       'additions', 'deletions']

In [ ]:
#create an empty dataframe and save it to a csv
if os.path.exists('/home/sreenath_a/Projects/OSS/Production/Julia_Commits.csv'):
    pass
else:
    df = pd.DataFrame(columns=columns)
    df.to_csv('/home/sreenath_a/Projects/OSS/Production/Julia_Commits.csv', index=False)

In [ ]:
#get all the packages alrady collected
slugs_collected = []
for df in pd.read_csv('/home/sreenath_a/Projects/OSS/Production/Julia_Commits.csv', chunksize=1000):
    repo_names = df['repo_name'].tolist()
    repo_owners = df['repo_owner'].tolist()
    for i in range(len(repo_names)):
        slugs_collected.append(repo_owners[i] + '/' + repo_names[i])
    slugs_collected = list(set(slugs_collected))

In [ ]:
keys_list = [(i.split(":")[0], i.split(":")[1]) for i in keys]
collector = collect(keys_list)

In [ ]:
parallelcount = cpu_count

In [ ]:
def get_owner_and_repo(url):
    ### if it has .git, remove it
    if url.endswith(".git"):
        url = url[:-4]
    ### get owner and repo
    owner = url.split("/")[-2]
    repo = url.split("/")[-1]
    return owner, repo

In [ ]:
def get_commits(owner, repo, key, since):
    try:
        if owner + '/' + repo in slugs_collected:
            return None
        commits = collector.collectCommits(owner, repo, key, since)
        return commits
    except Exception as e:
        print("Error: ", e)
        return None

In [ ]:
overall_it = int(pd.read_csv("/home/sreenath_a/Projects/OSS/nsf-oss/Data_Collection/Julia_data_collection/Julia_Packages_all.csv").shape[0]/parallelcount)

In [ ]:
for df in tqdm(pd.read_csv("/home/sreenath_a/Projects/OSS/nsf-oss/Data_Collection/Julia_data_collection/Julia_Packages_all.csv", chunksize=parallelcount), total=overall_it):
    slugs = [get_owner_and_repo(i) for i in df.RepoURL]
    owners = [i[0] for i in slugs]
    repos = [i[1] for i in slugs]
    best_keys = collector.getBestKeys(n=parallelcount)
    sinces = ["2007-01-01T00:00:00Z" for i in range(parallelcount)]
    args = list(zip(owners, repos, best_keys, sinces))
    with Pool(parallelcount) as p:
        result = p.starmap(get_commits, args)
    result = [i for i in result if i is not None]
    if len(result) == 0:
        continue
    df_result = pd.concat(result)[columns]
    df_result.to_csv('/home/sreenath_a/Projects/OSS/Production/Julia_Commits.csv', mode='a', header=False, index=False)